<a href="https://colab.research.google.com/github/Aniket1313/Deep-Learning-Practice/blob/master/Linear_Regression_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here we implement a simple linear regression model in Pytorch




In [0]:
import numpy as np
import torch


Training Data 

In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [0]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


**Linear Regression model**

In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.4997,  0.1192,  1.9132],
        [ 0.2235, -0.4668,  0.5094]], requires_grad=True)
tensor([1.4697, 0.4961], requires_grad=True)


In [0]:
#We now define the model

def model(x):
    return x @ w.t() + b

@ means multiplication and .t means transpose of tensor

In [7]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[128.1959,   7.4356],
        [179.8688,  12.3516],
        [171.8726, -13.0718],
        [128.3478,  22.0637],
        [181.3080,   6.7568]], grad_fn=<AddBackward0>)


In [8]:
print(targets)


tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


**Loss Function :MSE(Mean Squared Error)**

In [0]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()
#.sum returns sum of all elements in a tensor
#.numel returns no.of elements in a tensor

In [11]:
# Compute loss
loss = mse(preds, targets)
print(loss)


tensor(8115.6211, grad_fn=<DivBackward0>)



**Compute gradients**

In [0]:
# Compute gradients
loss.backward()

In [13]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.4997,  0.1192,  1.9132],
        [ 0.2235, -0.4668,  0.5094]], requires_grad=True)
tensor([[ 7023.5991,  6542.6050,  4383.0127],
        [-6922.1455, -8596.8193, -5049.1187]])


We need to reset gradients to zero because PyTorch accumulates, gradients i.e. the next time we call .backward on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.

In [14]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


**Adjust weights and biases using gradient descent**

Reduce the loss and improve our model using the gradient descent 
optimization algorithm, which has the following steps:

Generate predictions

Calculate the loss

Compute gradients w.r.t the weights and biases

1.   Generate predictions
2.   Calculate the loss
3.   Compute gradients w.r.t the weights and biases
4.   Adjust the weights by subtracting a small quantity proportional to the gradient.
5.   Reset the gradients to zero



Let's implement the above step by step.

In [15]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[128.1959,   7.4356],
        [179.8688,  12.3516],
        [171.8726, -13.0718],
        [128.3478,  22.0637],
        [181.3080,   6.7568]], grad_fn=<AddBackward0>)


In [16]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(8115.6211, grad_fn=<DivBackward0>)


In [17]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 7023.5991,  6542.6050,  4383.0127],
        [-6922.1455, -8596.8193, -5049.1187]])
tensor([ 81.7186, -84.8928])


In [0]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [19]:
print(w)
print(b)

tensor([[ 0.4294,  0.0537,  1.8693],
        [ 0.2927, -0.3808,  0.5598]], requires_grad=True)
tensor([1.4688, 0.4969], requires_grad=True)


In [20]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(5759.5444, grad_fn=<DivBackward0>)



**Train for multiple epochs**

In [0]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [22]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(292.6480, grad_fn=<DivBackward0>)


In [23]:
# Predictions
preds

tensor([[ 60.9594,  75.6090],
        [ 92.8314, 104.4813],
        [ 88.4343, 115.7718],
        [ 42.5249,  67.3585],
        [108.1783, 108.1590]], grad_fn=<AddBackward0>)

In [24]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])